In [ ]:
import math
import random
import json

# ------------ Settings ------------
min_lon, max_lon = 46.60, 46.85
min_lat, max_lat = 24.60, 24.80
cell_size_km = 1  # 1km grid

# Existing facilities
facilities = [
    {"name": "PHC Central", "region": "riyadh", "type": "phc", "priv": "gov", "coords": [46.7117, 24.7136], "population": 2000, "saudi": 1500, "insured": 1200},
    {"name": "Al Noor Hospital", "region": "riyadh", "type": "hospital", "priv": "gov", "coords": [46.7267, 24.713], "population": 3000, "saudi": 2400, "insured": 2500},
    {"name": "PHC West", "region": "riyadh", "type": "phc", "priv": "priv", "coords": [46.678, 24.721], "population": 1200, "saudi": 800, "insured": 1000},
    {"name": "Private Clinic", "region": "riyadh", "type": "phc", "priv": "priv", "coords": [46.68, 24.695], "population": 900, "saudi": 400, "insured": 700},
    {"name": "PHC South", "region": "riyadh", "type": "phc", "priv": "gov", "coords": [46.715, 24.68], "population": 1400, "saudi": 1100, "insured": 1000},
    {"name": "PHC Makkah", "region": "makkah", "type": "phc", "priv": "gov", "coords": [39.8262, 21.4225], "population": 2100, "saudi": 1600, "insured": 1700}
]

# Haversine distance in km
def haversine(lon1, lat1, lon2, lat2):
    R = 6371
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat/2)**2 + math.cos(math.radians(lat1))*math.cos(math.radians(lat2))*math.sin(dlon/2)**2
    return R * 2 * math.asin(math.sqrt(a))

# ----------- Step 1: Generate grid cells -----------
grid = []
lat = min_lat
while lat < max_lat:
    lon = min_lon
    while lon < max_lon:
        pop = random.randint(800, 4000)  # dummy population
        grid.append({"lon": lon, "lat": lat, "population": pop})
        lon += cell_size_km / 110
    lat += cell_size_km / 110

# ----------- Step 2: Calculate score for each cell -----------
for cell in grid:
    # Distance to nearest facility in same region (riyadh)
    distances = [
        haversine(cell['lon'], cell['lat'], fac['coords'][0], fac['coords'][1])
        for fac in facilities if fac["region"] == "riyadh"
    ]
    cell['min_dist_km'] = min(distances)
    cell['score'] = cell['population'] * cell['min_dist_km']

# ----------- Step 3: Top 3 cells -----------
top_cells = sorted(grid, key=lambda c: -c['score'])[:3]
best_locations = [
    {"name": f"Suggested Location {i+1}", "coords": [cell['lon'], cell['lat']]}
    for i, cell in enumerate(top_cells)
]

# ----------- Output as HTML -----------
html = f"""
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <title>Health Facility Map Dashboard with Best Locations</title>
  <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/ol@v9.2.4/ol.css">
  <style>
    body {{ margin:0; overflow:hidden; font-family:sans-serif; }}
    .Container {{ display: flex; height: 100vh; }}
    .Sidebar {{
      width: 310px; padding: 18px; background: #fafaff; border-right: 1px solid #ddd; box-sizing: border-box; overflow-y:auto;
    }}
    .Sidebar label, .Sidebar select, .Sidebar input[type=radio], .Sidebar input[type=range] {{
      display:block; margin-bottom:10px;
    }}
    .Sidebar input[type=radio] {{ margin-right: 7px; }}
    .Sidebar button {{ margin-top: 10px; width: 100%; padding: 8px; font-weight: bold; background: #4b58ff; color:white; border:none; border-radius:5px; cursor:pointer; }}
    .Sidebar h2 {{ margin:0 0 18px 0; }}
    .MapBody {{ flex:1; }}
    #map {{ width:100%; height:100%; }}
  </style>
</head>
<body>
<div class="Container">
  <div class="Sidebar">
    <h2>Lean</h2>
    <label for="region_select">Region</label>
    <select id="region_select">
      <option value="riyadh">Riyadh</option>
      <option value="makkah">Makkah</option>
    </select>
    <label for="type_select">Type</label>
    <select id="type_select">
      <option value="phc">PHC</option>
      <option value="hospital">Hospital</option>
    </select>
    <label for="private_select">Private/Government</label>
    <select id="private_select">
      <option value="all">All</option>
      <option value="gov">Government</option>
      <option value="priv">Private</option>
    </select>
    <label id="sliderAmount">Distance: <span id="bufferValue">500</span>m</label>
    <input type="range" min="500" max="3000" step="100" value="500" id="distance_slider" />
    <label>Display Metric</label>
    <label><input type="radio" name="metric_select" value="population" checked> Population</label>
    <label><input type="radio" name="metric_select" value="saudi"> Saudi Population</label>
    <label><input type="radio" name="metric_select" value="insured"> Insured</label>
    <button id="suggestBtn">Suggest Best Locations</button>
  </div>
  <div class="MapBody">
    <div id="map"></div>
  </div>
</div>
<script src="https://cdn.jsdelivr.net/npm/ol@v9.2.4/dist/ol.js"></script>
<script>
// --- Facilities ---
const facilities = {json.dumps(facilities, ensure_ascii=False, indent=None)};

// --- Suggested Locations ---
const bestLocations = {json.dumps(best_locations, ensure_ascii=False, indent=None)};

// --- OpenLayers Map setup ---
const map = new ol.Map({{
  target: 'map',
  layers: [
    new ol.layer.Tile({{ source: new ol.source.OSM() }})
  ],
  view: new ol.View({{
    center: ol.proj.fromLonLat([46.7117, 24.7136]), // Riyadh
    zoom: 12
  }})
}});

let vectorSource = new ol.source.Vector();
let vectorLayer = new ol.layer.Vector({{ source: vectorSource }});
map.addLayer(vectorLayer);

// Helper: Get selected filter values
function getFilters() {{
  return {{
    region: document.getElementById('region_select').value,
    type: document.getElementById('type_select').value,
    priv: document.getElementById('private_select').value,
    metric: document.querySelector('input[name="metric_select"]:checked').value,
    buffer: parseInt(document.getElementById('distance_slider').value)
  }};
}}

// Render facilities/buffers (like before)
function renderFacilities() {{
  const filters = getFilters();
  vectorSource.clear();

  // Filter facilities by region, type, and ownership
  let shown = facilities.filter(f =>
    f.region === filters.region &&
    (filters.type === 'phc' ? f.type === 'phc' : f.type === filters.type) &&
    (filters.priv === 'all' || (filters.priv === 'gov' ? f.priv === 'gov' : f.priv === 'priv'))
  );

  // Add buffer and marker for each facility
  shown.forEach(fac => {{
    // Buffer (circle)
    let circle = new ol.Feature({{
      geometry: new ol.geom.Circle(ol.proj.fromLonLat(fac.coords), filters.buffer)
    }});
    circle.setStyle(new ol.style.Style({{
      stroke: new ol.style.Stroke({{ color: '#4b58ffAA', width: 2 }}),
      fill: new ol.style.Fill({{ color: '#4b58ff22' }})
    }}));
    vectorSource.addFeature(circle);

    // Marker
    let marker = new ol.Feature({{
      geometry: new ol.geom.Point(ol.proj.fromLonLat(fac.coords)),
      name: fac.name
    }});

    // Color and radius based on metric
    let val = fac[filters.metric] || 0;
    let radius = Math.max(8, Math.min(20, val/200)); // scale marker
    let color = filters.metric === "population" ? "#ff5d5d" : (filters.metric === "saudi" ? "#33b249" : "#2e85ff");
    marker.setStyle(new ol.style.Style({{
      image: new ol.style.Circle({{
        radius: radius,
        fill: new ol.style.Fill({{ color }}),
        stroke: new ol.style.Stroke({{ color: '#222', width: 1 }})
      }}),
      text: new ol.style.Text({{
        text: fac.name + '\\n' + val,
        font: 'bold 12px Arial',
        offsetY: -22,
        fill: new ol.style.Fill({{ color: "#444" }}),
        stroke: new ol.style.Stroke({{ color: "#fff", width: 2 }})
      }})
    }}));
    vectorSource.addFeature(marker);
  }});
}}

// Show best locations (on suggest button click)
function showBestLocations() {{
  // Remove old suggestion markers first if any
  renderFacilities();

  bestLocations.forEach((loc, i) => {{
    let marker = new ol.Feature({{
      geometry: new ol.geom.Point(ol.proj.fromLonLat(loc.coords)),
      name: loc.name
    }});
    marker.setStyle(new ol.style.Style({{
      image: new ol.style.Circle({{
        radius: 18,
        fill: new ol.style.Fill({{ color: '#FFD600' }}),
        stroke: new ol.style.Stroke({{ color: '#FF6D00', width: 3 }})
      }}),
      text: new ol.style.Text({{
        text: loc.name,
        font: 'bold 13px Arial',
        offsetY: -26,
        fill: new ol.style.Fill({{ color: "#444" }}),
        stroke: new ol.style.Stroke({{ color: "#fff", width: 2 }})
      }})
    }}));
    vectorSource.addFeature(marker);
  }});
}}

// --- UI event listeners ---
document.getElementById('region_select').onchange =
document.getElementById('type_select').onchange =
document.getElementById('private_select').onchange =
document.querySelectorAll('input[name="metric_select"]').forEach(el => el.onchange = renderFacilities);
document.getElementById('distance_slider').oninput = function() {{
  document.getElementById('bufferValue').textContent = this.value;
  renderFacilities();
}};
document.getElementById('suggestBtn').onclick = showBestLocations;

// Initial render
renderFacilities();
</script>
</body>
</html>
"""

with open("dashboard_best_locations.html", "w", encoding="utf-8") as f:
    f.write(html)

print("dashboard_best_locations.html generated! Open it in your browser.")


In [ ]:
#keep ryidh pop only

import geopandas as gpd
gdf = gpd.read_file("population_grid_with_geometry.geojson")
riyadh_gdf = gdf.cx[46.5:47, 24.5:25]  # example bounds for Riyadh
riyadh_gdf.to_file("pop_grid_riyadh.geojson", driver="GeoJSON")

In [ ]:
import json

facilities = [
    {"name": "PHC Central", "coords": [46.7117, 24.7136]},
    {"name": "Al Noor Hospital", "coords": [46.7267, 24.713]},
]

# Load your new Riyadh population grid (3.7 MB GeoJSON)
with open("pop_grid_riyadh.geojson", encoding="utf-8") as f:
    pop_grid = json.load(f)

html = """
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <title>Lean Facility Dashboard + Riyadh Population</title>
  <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/ol@v9.2.4/ol.css">
  <style>
    html, body {{
      height: 100%;
      margin: 0;
      padding: 0;
    }}
    body {{
      font-family: sans-serif;
      height: 100%;
      margin: 0;
      padding: 0;
      overflow: hidden;
    }}
    .Container {{
      display: flex;
      height: 100vh;
    }}
    .Sidebar {{
      width: 200px;
      background: #fafaff;
      border-right: 1px solid #ddd;
      padding: 20px;
      box-sizing: border-box;
    }}
    .MapBody {{
      flex: 1;
      height: 100vh;
      min-width: 300px;
      position: relative;
    }}
    #map {{
      width: 100%;
      height: 100vh;
      min-height: 600px;
      min-width: 400px;
      background: #eee;
    }}
  </style>
</head>
<body>
<div class="Container">
  <div class="Sidebar">
    <h2>Lean</h2>
    <p>Population grid (colored by total population in each 400m cell).</p>
  </div>
  <div class="MapBody">
    <div id="map"></div>
  </div>
</div>
<script src="https://cdn.jsdelivr.net/npm/ol@v9.2.4/dist/ol.js"></script>
<script>
  var facilities = {facilities};
  var popGrid = {pop_grid};

  // Map
  var map = new ol.Map({{
    target: 'map',
    layers: [
      new ol.layer.Tile({{ source: new ol.source.OSM() }})
    ],
    view: new ol.View({{
      center: ol.proj.fromLonLat([46.7117, 24.7136]), // Center on Riyadh
      zoom: 11
    }})
  }});

  // --- Add Population Grid Polygons ---
  function getColor(pop) {{
    if (pop == null || pop < 1) return 'rgba(0,0,0,0)';
    if (pop > 1000) return '#b71c1c';    // Deep red
    if (pop > 500)  return '#ef5350';    // Red
    if (pop > 250)  return '#ffb300';    // Orange
    if (pop > 100)  return '#fff176';    // Light yellow
    return '#fffde7';                    // Pale yellow
  }}

  var popFeatures = new ol.format.GeoJSON().readFeatures(popGrid, {{
    featureProjection: 'EPSG:3857',
    dataProjection: 'EPSG:4326'
  }});
  popFeatures.forEach(function(f) {{
    var pop = f.get('PCNT') || f.get('pop') || 0;
    f.setStyle(new ol.style.Style({{
      stroke: new ol.style.Stroke({{color: '#666', width: 0.2}}),
      fill: new ol.style.Fill({{color: getColor(pop), opacity: 0.5}})
    }}));
  }});
  var popLayer = new ol.layer.Vector({{
    source: new ol.source.Vector({{
      features: popFeatures
    }}),
    opacity: 0.6
  }});
  map.addLayer(popLayer);

  // --- Facility Markers ---
  var features = facilities.map(function(fac) {{
    var marker = new ol.Feature({{
      geometry: new ol.geom.Point(ol.proj.fromLonLat(fac.coords))
    }});
    marker.setStyle(new ol.style.Style({{
      image: new ol.style.Circle({{
        radius: 10,
        fill: new ol.style.Fill({{color: '#E53935'}}),
        stroke: new ol.style.Stroke({{color: '#fff', width: 2}})
      }}),
      text: new ol.style.Text({{
        text: fac.name,
        offsetY: -18,
        font: 'bold 13px Arial',
        fill: new ol.style.Fill({{color: '#222'}}),
        stroke: new ol.style.Stroke({{color: '#fff', width: 2}})
      }})
    }}));
    return marker;
  }});

  var vectorSource = new ol.source.Vector({{
    features: features
  }});
  var vectorLayer = new ol.layer.Vector({{
    source: vectorSource
  }});
  map.addLayer(vectorLayer);
</script>
</body>
</html>
""".format(facilities=json.dumps(facilities, ensure_ascii=False), pop_grid=json.dumps(pop_grid))

with open("dashboard_riyadh_population.html", "w", encoding="utf-8") as f:
    f.write(html)

print("dashboard_riyadh_population.html generated! Open it in your browser.")


In [ ]:
import json

facilities = [
    {"name": "PHC Central", "coords": [46.7117, 24.7136]},
    {"name": "Al Noor Hospital", "coords": [46.7267, 24.713]},
]

# Load the grid GeoJSON file (use your file path)
with open("pop_grid_riyadh.geojson", encoding="utf-8") as f:
    pop_grid = json.load(f)

html = """
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <title>Lean Facility Dashboard + Riyadh Population</title>
  <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/ol@v9.2.4/ol.css">
  <style>
    html, body {{
      height: 100%;
      margin: 0;
      padding: 0;
    }}
    body {{
      font-family: sans-serif;
      height: 100%;
      margin: 0;
      padding: 0;
      overflow: hidden;
    }}
    .Container {{
      display: flex;
      height: 100vh;
    }}
    .Sidebar {{
      width: 200px;
      background: #fafaff;
      border-right: 1px solid #ddd;
      padding: 20px;
      box-sizing: border-box;
    }}
    .MapBody {{
      flex: 1;
      height: 100vh;
      min-width: 300px;
      position: relative;
    }}
    #map {{
      width: 100%;
      height: 100vh;
      min-height: 600px;
      min-width: 400px;
      background: #eee;
    }}
    .ol-popup {{
      position: absolute;
      background-color: white;
      box-shadow: 0 1px 4px rgba(0,0,0,0.2);
      padding: 10px;
      border-radius: 6px;
      border: 1px solid #cccccc;
      min-width: 200px;
      bottom: 12px;
      left: -50px;
      font-size: 14px;
      z-index: 999;
    }}
    .ol-popup:after, .ol-popup:before {{
      top: 100%;
      border: solid transparent;
      content: " ";
      height: 0;
      width: 0;
      position: absolute;
      pointer-events: none;
    }}
    .ol-popup:after {{
      border-top-color: white;
      border-width: 10px;
      left: 48px;
      margin-left: -10px;
    }}
    .ol-popup:before {{
      border-top-color: #cccccc;
      border-width: 11px;
      left: 48px;
      margin-left: -11px;
    }}
  </style>
</head>
<body>
<div class="Container">
  <div class="Sidebar">
    <h2>Lean</h2>
    <p>Population grid (hover cells for details)</p>
  </div>
  <div class="MapBody">
    <div id="map"></div>
    <div id="popup" class="ol-popup" style="display:none;"></div>
  </div>
</div>
<script src="https://cdn.jsdelivr.net/npm/ol@v9.2.4/dist/ol.js"></script>
<script>
  var facilities = {facilities};
  var popGrid = {pop_grid};

  // Map
  var map = new ol.Map({{
    target: 'map',
    layers: [
      new ol.layer.Tile({{ source: new ol.source.OSM() }})
    ],
    view: new ol.View({{
      center: ol.proj.fromLonLat([46.7117, 24.7136]),
      zoom: 11
    }})
  }});

  // --- Add Population Grid Polygons ---
  function getColor(pop) {{
    if (pop == null || pop < 1) return 'rgba(0,0,0,0)';
    if (pop > 1000) return '#b71c1c';    // Deep red
    if (pop > 500)  return '#ef5350';    // Red
    if (pop > 250)  return '#ffb300';    // Orange
    if (pop > 100)  return '#fff176';    // Light yellow
    return '#fffde7';                    // Pale yellow
  }}

  var popFeatures = new ol.format.GeoJSON().readFeatures(popGrid, {{
    featureProjection: 'EPSG:3857',
    dataProjection: 'EPSG:4326'
  }});
  popFeatures.forEach(function(f) {{
    var pop = f.get('PCNT') || f.get('pop') || 0;
    f.setStyle(new ol.style.Style({{
      stroke: new ol.style.Stroke({{color: '#666', width: 0.2}}),
      fill: new ol.style.Fill({{color: getColor(pop), opacity: 0.5}})
    }}));
  }});
  var popLayer = new ol.layer.Vector({{
    source: new ol.source.Vector({{
      features: popFeatures
    }}),
    opacity: 0.6
  }});
  map.addLayer(popLayer);

  // --- Facility Markers ---
  var features = facilities.map(function(fac) {{
    var marker = new ol.Feature({{
      geometry: new ol.geom.Point(ol.proj.fromLonLat(fac.coords))
    }});
    marker.setStyle(new ol.style.Style({{
      image: new ol.style.Circle({{
        radius: 10,
        fill: new ol.style.Fill({{color: '#E53935'}}),
        stroke: new ol.style.Stroke({{color: '#fff', width: 2}})
      }}),
      text: new ol.style.Text({{
        text: fac.name,
        offsetY: -18,
        font: 'bold 13px Arial',
        fill: new ol.style.Fill({{color: '#222'}}),
        stroke: new ol.style.Stroke({{color: '#fff', width: 2}})
      }})
    }}));
    return marker;
  }});

  var vectorSource = new ol.source.Vector({{
    features: features
  }});
  var vectorLayer = new ol.layer.Vector({{
    source: vectorSource
  }});
  map.addLayer(vectorLayer);

  // --- Popup for Population Cell Info ---
  var popup = document.getElementById('popup');
  var overlay = new ol.Overlay({{
    element: popup,
    positioning: 'bottom-center',
    stopEvent: false,
    offset: [0, -8]
  }});
  map.addOverlay(overlay);

  map.on('pointermove', function(evt) {{
    var feature = map.forEachFeatureAtPixel(evt.pixel, function(feature, layer) {{
      // Only show popup for grid polygons, not markers
      if (layer === popLayer) return feature;
    }});
    if (feature) {{
      var p = feature.getProperties();
      // Exclude the geometry property from the output
      var html = "<b>Population:</b> " + (p.PCNT ?? 'N/A') + "<br>" +
                 "<b>Male:</b> " + (p.PM_CNT ?? 'N/A') + "<br>" +
                 "<b>Female:</b> " + (p.PF_CNT ?? 'N/A') + "<br>" +
                 "<b>Density/km²:</b> " + (p.PDEN_KM2 ?? 'N/A') + "<br>" +
                 "<b>Median Age:</b> " + (p.YMED_AGE ?? 'N/A') + "<br>" +
                 "<b>Median Age (M):</b> " + (p.YMED_AGE_M ?? 'N/A') + "<br>" +
                 "<b>Median Age (F):</b> " + (p.YMED_AGE_FM ?? 'N/A');
      popup.innerHTML = html;
      popup.style.display = 'block';
      overlay.setPosition(evt.coordinate);
    }} else {{
      popup.style.display = 'none';
    }}
  }});
</script>
</body>
</html>
""".format(facilities=json.dumps(facilities, ensure_ascii=False), pop_grid=json.dumps(pop_grid))

with open("dashboard_riyadh_population_popup.html", "w", encoding="utf-8") as f:
    f.write(html)

print("dashboard_riyadh_population_popup.html generated! Open it in your browser.")


In [ ]:
import json

facilities = [
    {"name": "PHC Central", "coords": [46.7117, 24.7136]},
    {"name": "Al Noor Hospital", "coords": [46.7267, 24.713]},
]

with open("pop_grid_riyadh.geojson", encoding="utf-8") as f:
    pop_grid = json.load(f)

html = """
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <title>Lean Facility Dashboard + Riyadh Population</title>
  <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/ol@v9.2.4/ol.css">
  <style>
    html, body {{
      height: 100%;
      margin: 0;
      padding: 0;
    }}
    body {{
      font-family: sans-serif;
      height: 100%;
      margin: 0;
      padding: 0;
      overflow: hidden;
    }}
    .Container {{
      display: flex;
      height: 100vh;
    }}
    .Sidebar {{
      width: 270px;
      background: #fafaff;
      border-right: 1px solid #ddd;
      padding: 22px 15px 15px 15px;
      box-sizing: border-box;
      font-size: 15px;
    }}
    .Sidebar label, .Sidebar select, .Sidebar input[type=number] {{
      margin-bottom: 12px;
      display: block;
      width: 100%;
    }}
    .MapBody {{
      flex: 1;
      height: 100vh;
      min-width: 300px;
      position: relative;
    }}
    #map {{
      width: 100%;
      height: 100vh;
      min-height: 600px;
      min-width: 400px;
      background: #eee;
    }}
    .ol-popup {{
      position: absolute;
      background-color: white;
      box-shadow: 0 1px 4px rgba(0,0,0,0.2);
      padding: 10px;
      border-radius: 6px;
      border: 1px solid #cccccc;
      min-width: 210px;
      bottom: 12px;
      left: -50px;
      font-size: 14px;
      z-index: 999;
      pointer-events: none;
    }}
  </style>
</head>
<body>
<div class="Container">
  <div class="Sidebar">
    <h2>Lean</h2>
    <label for="metric_select"><b>Color By Metric</b></label>
    <select id="metric_select">
      <option value="PCNT">Total Population</option>
      <option value="PM_CNT">Male Population</option>
      <option value="PF_CNT">Female Population</option>
      <option value="PDEN_KM2">Density (per km²)</option>
      <option value="YMED_AGE">Median Age</option>
      <option value="YMED_AGE_M">Median Age (M)</option>
      <option value="YMED_AGE_FM">Median Age (F)</option>
    </select>
    <label for="minval"><b>Filter: Show only cells above this value</b></label>
    <input type="number" id="minval" value="0" min="0" step="1">
    <button id="applyBtn" style="margin-top:10px;width:100%;padding:8px;font-weight:bold;background:#3f51b5;color:white;border:none;border-radius:5px;cursor:pointer;">Apply</button>
    <hr>
    <p style="font-size:14px;">Move the mouse over any cell to see its stats.<br>Facility locations shown as red dots.</p>
  </div>
  <div class="MapBody">
    <div id="map"></div>
    <div id="popup" class="ol-popup" style="display:none;"></div>
  </div>
</div>
<script src="https://cdn.jsdelivr.net/npm/ol@v9.2.4/dist/ol.js"></script>
<script>
  var facilities = {facilities};
  var popGrid = {pop_grid};

  var metric = 'PCNT';
  var minval = 0;

  var map = new ol.Map({{
    target: 'map',
    layers: [
      new ol.layer.Tile({{ source: new ol.source.OSM() }})
    ],
    view: new ol.View({{
      center: ol.proj.fromLonLat([46.7117, 24.7136]),
      zoom: 11
    }})
  }});

  // --- Population Grid Layer ---
  var popFeatures_all = new ol.format.GeoJSON().readFeatures(popGrid, {{
    featureProjection: 'EPSG:3857',
    dataProjection: 'EPSG:4326'
  }});

  function getColor(value, which) {{
    // color ramps by metric (simple: red-to-yellow)
    if (value == null || value === '' || value < minval) return 'rgba(0,0,0,0)';
    var v = Number(value);
    if (which.startsWith("YMED_AGE")) {{
      // Age: blue for young, red for old
      if (v >= 50) return "#6d0e0e";
      if (v >= 40) return "#b71c1c";
      if (v >= 30) return "#ef5350";
      if (v >= 20) return "#ffb300";
      if (v >= 10) return "#fff176";
      return "#fffde7";
    }} else if (which.includes("DEN")) {{
      // Density: darker = denser
      if (v > 5000) return "#7f0000";
      if (v > 2000) return "#b71c1c";
      if (v > 1000) return "#ef5350";
      if (v > 500)  return "#ffb300";
      if (v > 200)  return "#fff176";
      return "#fffde7";
    }} else {{
      // Population: red ramp
      if (v > 1000) return "#b71c1c";
      if (v > 500)  return "#ef5350";
      if (v > 250)  return "#ffb300";
      if (v > 100)  return "#fff176";
      return "#fffde7";
    }}
  }}

  var popLayer = new ol.layer.Vector({{
    source: new ol.source.Vector(),
    opacity: 0.6
  }});
  map.addLayer(popLayer);

  function styleFeature(f) {{
    var val = f.get(metric);
    return new ol.style.Style({{
      stroke: new ol.style.Stroke({{color: '#666', width: 0.2}}),
      fill: new ol.style.Fill({{color: getColor(val, metric), opacity: 0.5}})
    }});
  }}

  function renderPopLayer() {{
    popLayer.getSource().clear();
    popFeatures_all.forEach(function(f) {{
      var v = f.get(metric);
      // Hide polygons that don't pass the filter
      if (v != null && v !== '' && Number(v) >= minval)
        f.setStyle(styleFeature(f));
      else
        f.setStyle(new ol.style.Style({{fill: new ol.style.Fill({{color:'rgba(0,0,0,0)'}})}}));
    }});
    popLayer.getSource().addFeatures(popFeatures_all);
  }}

  // Facility Markers
  var features = facilities.map(function(fac) {{
    var marker = new ol.Feature({{
      geometry: new ol.geom.Point(ol.proj.fromLonLat(fac.coords))
    }});
    marker.setStyle(new ol.style.Style({{
      image: new ol.style.Circle({{
        radius: 10,
        fill: new ol.style.Fill({{color: '#E53935'}}),
        stroke: new ol.style.Stroke({{color: '#fff', width: 2}})
      }}),
      text: new ol.style.Text({{
        text: fac.name,
        offsetY: -18,
        font: 'bold 13px Arial',
        fill: new ol.style.Fill({{color: '#222'}}),
        stroke: new ol.style.Stroke({{color: '#fff', width: 2}})
      }})
    }}));
    return marker;
  }});
  var vectorSource = new ol.source.Vector({{ features: features }});
  var vectorLayer = new ol.layer.Vector({{ source: vectorSource }});
  map.addLayer(vectorLayer);

  // Popup for Population Cell Info
  var popup = document.getElementById('popup');
  var overlay = new ol.Overlay({{
    element: popup,
    positioning: 'bottom-center',
    stopEvent: false,
    offset: [0, -8]
  }});
  map.addOverlay(overlay);

  map.on('pointermove', function(evt) {{
    var feature = map.forEachFeatureAtPixel(evt.pixel, function(feature, layer) {{
      if (layer === popLayer) return feature;
    }});
    if (feature) {{
      var p = feature.getProperties();
      var html = "<b>Population:</b> " + (p.PCNT ?? 'N/A') + "<br>" +
                 "<b>Male:</b> " + (p.PM_CNT ?? 'N/A') + "<br>" +
                 "<b>Female:</b> " + (p.PF_CNT ?? 'N/A') + "<br>" +
                 "<b>Density/km²:</b> " + (p.PDEN_KM2 ?? 'N/A') + "<br>" +
                 "<b>Median Age:</b> " + (p.YMED_AGE ?? 'N/A') + "<br>" +
                 "<b>Median Age (M):</b> " + (p.YMED_AGE_M ?? 'N/A') + "<br>" +
                 "<b>Median Age (F):</b> " + (p.YMED_AGE_FM ?? 'N/A');
      popup.innerHTML = html;
      popup.style.display = 'block';
      overlay.setPosition(evt.coordinate);
    }} else {{
      popup.style.display = 'none';
    }}
  }});

  // UI controls
  document.getElementById('applyBtn').onclick = function() {{
    metric = document.getElementById('metric_select').value;
    minval = Number(document.getElementById('minval').value) || 0;
    renderPopLayer();
  }};
  // Initial render
  renderPopLayer();

</script>
</body>
</html>
""".format(facilities=json.dumps(facilities, ensure_ascii=False), pop_grid=json.dumps(pop_grid))

with open("dashboard_riyadh_population_filter.html", "w", encoding="utf-8") as f:
    f.write(html)

print("dashboard_riyadh_population_filter.html generated! Open it in your browser.")


In [ ]:
#random healthcare facilities as dummy data
# 
import pandas as pd
import random

# Riyadh bounding box (approx)
min_lon, max_lon = 46.55, 47.10
min_lat, max_lat = 24.40, 25.00

# Facility types
facility_types = ['Hospital', 'PHC', 'Pharmacy', 'Clinic', 'Polyclinic', 'Lab', 'Dental Center', 'Imaging Center']

data = []
for i in range(1, 501):
    name = f"Facility_{i:03d}"
    lon = round(random.uniform(min_lon, max_lon), 6)
    lat = round(random.uniform(min_lat, max_lat), 6)
    fac_type = random.choice(facility_types)
    data.append({'Name': name, 'Longitude': lon, 'Latitude': lat, 'Type': fac_type})

df = pd.DataFrame(data)
df.to_excel("random_riyadh_facilities.xlsx", index=False)
print("Generated 'random_riyadh_facilities.xlsx' with 500 facilities. Open it in Excel!")


In [ ]:
import pandas as pd
import json

# Load the 500-facility Excel file
df = pd.read_excel("random_riyadh_facilities.xlsx")

# Convert to the list of dicts for the dashboard
facilities = []
for _, row in df.iterrows():
    facilities.append({
        "name": row['Name'],
        "coords": [row['Longitude'], row['Latitude']],
        "type": row['Type']
    })

# Save as JSON (optional, for debugging)
with open("random_facilities.json", "w", encoding="utf-8") as f:
    json.dump(facilities, f, ensure_ascii=False, indent=2)


In [ ]:
with open("pop_grid_riyadh.geojson", encoding="utf-8") as f:
    pop_grid = json.load(f)

html = """
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <title>Lean Facility Dashboard + Riyadh Population</title>
  <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/ol@v9.2.4/ol.css">
  <style>
    html, body {{
      height: 100%;
      margin: 0;
      padding: 0;
    }}
    body {{
      font-family: sans-serif;
      height: 100%;
      margin: 0;
      padding: 0;
      overflow: hidden;
    }}
    .Container {{
      display: flex;
      height: 100vh;
    }}
    .Sidebar {{
      width: 270px;
      background: #fafaff;
      border-right: 1px solid #ddd;
      padding: 22px 15px 15px 15px;
      box-sizing: border-box;
      font-size: 15px;
    }}
    .Sidebar label, .Sidebar select, .Sidebar input[type=number] {{
      margin-bottom: 12px;
      display: block;
      width: 100%;
    }}
    .MapBody {{
      flex: 1;
      height: 100vh;
      min-width: 300px;
      position: relative;
    }}
    #map {{
      width: 100%;
      height: 100vh;
      min-height: 600px;
      min-width: 400px;
      background: #eee;
    }}
    .ol-popup {{
      position: absolute;
      background-color: white;
      box-shadow: 0 1px 4px rgba(0,0,0,0.2);
      padding: 10px;
      border-radius: 6px;
      border: 1px solid #cccccc;
      min-width: 210px;
      bottom: 12px;
      left: -50px;
      font-size: 14px;
      z-index: 999;
      pointer-events: none;
    }}
  </style>
</head>
<body>
<div class="Container">
  <div class="Sidebar">
    <h2>Lean</h2>
    <label for="metric_select"><b>Color By Metric</b></label>
    <select id="metric_select">
      <option value="PCNT">Total Population</option>
      <option value="PM_CNT">Male Population</option>
      <option value="PF_CNT">Female Population</option>
      <option value="PDEN_KM2">Density (per km²)</option>
      <option value="YMED_AGE">Median Age</option>
      <option value="YMED_AGE_M">Median Age (M)</option>
      <option value="YMED_AGE_FM">Median Age (F)</option>
    </select>
    <label for="minval"><b>Filter: Show only cells above this value</b></label>
    <input type="number" id="minval" value="0" min="0" step="1">
    <button id="applyBtn" style="margin-top:10px;width:100%;padding:8px;font-weight:bold;background:#3f51b5;color:white;border:none;border-radius:5px;cursor:pointer;">Apply</button>
    <hr>
    <p style="font-size:14px;">Move the mouse over any cell to see its stats.<br>Facility locations shown as colored dots.</p>
  </div>
  <div class="MapBody">
    <div id="map"></div>
    <div id="popup" class="ol-popup" style="display:none;"></div>
  </div>
</div>
<script src="https://cdn.jsdelivr.net/npm/ol@v9.2.4/dist/ol.js"></script>
<script>
  var facilities = {facilities};
  var popGrid = {pop_grid};

  var typeColors = {{
    "Hospital": "#D32F2F",
    "PHC": "#388E3C",
    "Pharmacy": "#1976D2",
    "Clinic": "#FFA000",
    "Polyclinic": "#7B1FA2",
    "Lab": "#0288D1",
    "Dental Center": "#C2185B",
    "Imaging Center": "#512DA8"
  }};

  var metric = 'PCNT';
  var minval = 0;

  var map = new ol.Map({{
    target: 'map',
    layers: [
      new ol.layer.Tile({{ source: new ol.source.OSM() }})
    ],
    view: new ol.View({{
      center: ol.proj.fromLonLat([46.7117, 24.7136]),
      zoom: 11
    }})
  }});

  // --- Population Grid Layer ---
  var popFeatures_all = new ol.format.GeoJSON().readFeatures(popGrid, {{
    featureProjection: 'EPSG:3857',
    dataProjection: 'EPSG:4326'
  }});

  function getColor(value, which) {{
    // color ramps by metric (simple: red-to-yellow)
    if (value == null || value === '' || value < minval) return 'rgba(0,0,0,0)';
    var v = Number(value);
    if (which.startsWith("YMED_AGE")) {{
      if (v >= 50) return "#6d0e0e";
      if (v >= 40) return "#b71c1c";
      if (v >= 30) return "#ef5350";
      if (v >= 20) return "#ffb300";
      if (v >= 10) return "#fff176";
      return "#fffde7";
    }} else if (which.includes("DEN")) {{
      if (v > 5000) return "#7f0000";
      if (v > 2000) return "#b71c1c";
      if (v > 1000) return "#ef5350";
      if (v > 500)  return "#ffb300";
      if (v > 200)  return "#fff176";
      return "#fffde7";
    }} else {{
      if (v > 1000) return "#b71c1c";
      if (v > 500)  return "#ef5350";
      if (v > 250)  return "#ffb300";
      if (v > 100)  return "#fff176";
      return "#fffde7";
    }}
  }}

  var popLayer = new ol.layer.Vector({{
    source: new ol.source.Vector(),
    opacity: 0.6
  }});
  map.addLayer(popLayer);

  function styleFeature(f) {{
    var val = f.get(metric);
    return new ol.style.Style({{
      stroke: new ol.style.Stroke({{color: '#666', width: 0.2}}),
      fill: new ol.style.Fill({{color: getColor(val, metric), opacity: 0.5}})
    }});
  }}

  function renderPopLayer() {{
    popLayer.getSource().clear();
    popFeatures_all.forEach(function(f) {{
      var v = f.get(metric);
      if (v != null && v !== '' && Number(v) >= minval)
        f.setStyle(styleFeature(f));
      else
        f.setStyle(new ol.style.Style({{fill: new ol.style.Fill({{color:'rgba(0,0,0,0)'}})}}));
    }});
    popLayer.getSource().addFeatures(popFeatures_all);
  }}

  // Facility Markers (500 random, colored by type)
  var features = facilities.map(function(fac) {{
    var color = typeColors[fac.type] || "#222";
    var marker = new ol.Feature({{
      geometry: new ol.geom.Point(ol.proj.fromLonLat(fac.coords))
    }});
    marker.setStyle(new ol.style.Style({{
      image: new ol.style.Circle({{
        radius: 7,
        fill: new ol.style.Fill({{color: color}}),
        stroke: new ol.style.Stroke({{color: '#fff', width: 1}})
      }}),
      text: new ol.style.Text({{
        text: fac.name,
        offsetY: -16,
        font: 'bold 10px Arial',
        fill: new ol.style.Fill({{color: "#444"}}),
        stroke: new ol.style.Stroke({{color: "#fff", width: 2}})
      }})
    }}));
    return marker;
  }});
  var vectorSource = new ol.source.Vector({{ features: features }});
  var vectorLayer = new ol.layer.Vector({{ source: vectorSource }});
  map.addLayer(vectorLayer);

  // Popup for Population Cell Info
  var popup = document.getElementById('popup');
  var overlay = new ol.Overlay({{
    element: popup,
    positioning: 'bottom-center',
    stopEvent: false,
    offset: [0, -8]
  }});
  map.addOverlay(overlay);

  map.on('pointermove', function(evt) {{
    var feature = map.forEachFeatureAtPixel(evt.pixel, function(feature, layer) {{
      if (layer === popLayer) return feature;
    }});
    if (feature) {{
      var p = feature.getProperties();
      var html = "<b>Population:</b> " + (p.PCNT ?? 'N/A') + "<br>" +
                 "<b>Male:</b> " + (p.PM_CNT ?? 'N/A') + "<br>" +
                 "<b>Female:</b> " + (p.PF_CNT ?? 'N/A') + "<br>" +
                 "<b>Density/km²:</b> " + (p.PDEN_KM2 ?? 'N/A') + "<br>" +
                 "<b>Median Age:</b> " + (p.YMED_AGE ?? 'N/A') + "<br>" +
                 "<b>Median Age (M):</b> " + (p.YMED_AGE_M ?? 'N/A') + "<br>" +
                 "<b>Median Age (F):</b> " + (p.YMED_AGE_FM ?? 'N/A');
      popup.innerHTML = html;
      popup.style.display = 'block';
      overlay.setPosition(evt.coordinate);
    }} else {{
      popup.style.display = 'none';
    }}
  }});

  // UI controls
  document.getElementById('applyBtn').onclick = function() {{
    metric = document.getElementById('metric_select').value;
    minval = Number(document.getElementById('minval').value) || 0;
    renderPopLayer();
  }};
  // Initial render
  renderPopLayer();
</script>
</body>
</html>
""".format(
    facilities=json.dumps(facilities, ensure_ascii=False),
    pop_grid=json.dumps(pop_grid)
)

with open("dashboard_riyadh_population_facilities.html", "w", encoding="utf-8") as f:
    f.write(html)

print("dashboard_riyadh_population_facilities.html generated! Open it in your browser.")


In [ ]:
import pandas as pd
import json

# Load facilities from your Excel
df = pd.read_excel("random_riyadh_facilities.xlsx")
facilities = []
for _, row in df.iterrows():
    facilities.append({
        "name": row['Name'],
        "coords": [row['Longitude'], row['Latitude']],
        "type": row['Type']
    })

with open("pop_grid_riyadh.geojson", encoding="utf-8") as f:
    pop_grid = json.load(f)

# Get the unique types for the dropdown
facility_types = sorted(df['Type'].unique())
facility_type_options = "\n".join(
    f'<option value="{ft}">{ft}</option>' for ft in facility_types
)

html = f"""
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <title>Lean Facility Dashboard + Riyadh Population</title>
  <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/ol@v9.2.4/ol.css">
  <style>
    html, body {{
      height: 100%;
      margin: 0;
      padding: 0;
    }}
    body {{
      font-family: sans-serif;
      height: 100%;
      margin: 0;
      padding: 0;
      overflow: hidden;
    }}
    .Container {{
      display: flex;
      height: 100vh;
    }}
    .Sidebar {{
      width: 270px;
      background: #fafaff;
      border-right: 1px solid #ddd;
      padding: 22px 15px 15px 15px;
      box-sizing: border-box;
      font-size: 15px;
    }}
    .Sidebar label, .Sidebar select, .Sidebar input[type=number] {{
      margin-bottom: 12px;
      display: block;
      width: 100%;
    }}
    .MapBody {{
      flex: 1;
      height: 100vh;
      min-width: 300px;
      position: relative;
    }}
    #map {{
      width: 100%;
      height: 100vh;
      min-height: 600px;
      min-width: 400px;
      background: #eee;
    }}
    .ol-popup {{
      position: absolute;
      background-color: white;
      box-shadow: 0 1px 4px rgba(0,0,0,0.2);
      padding: 10px;
      border-radius: 6px;
      border: 1px solid #cccccc;
      min-width: 210px;
      bottom: 12px;
      left: -50px;
      font-size: 14px;
      z-index: 999;
      pointer-events: none;
    }}
  </style>
</head>
<body>
<div class="Container">
  <div class="Sidebar">
    <h2>Lean</h2>
    <label for="metric_select"><b>Color By Metric</b></label>
    <select id="metric_select">
      <option value="PCNT">Total Population</option>
      <option value="PM_CNT">Male Population</option>
      <option value="PF_CNT">Female Population</option>
      <option value="PDEN_KM2">Density (per km²)</option>
      <option value="YMED_AGE">Median Age</option>
      <option value="YMED_AGE_M">Median Age (M)</option>
      <option value="YMED_AGE_FM">Median Age (F)</option>
    </select>
    <label for="minval"><b>Filter: Show only cells above this value</b></label>
    <input type="number" id="minval" value="0" min="0" step="1">

    <label for="facility_type"><b>Show Facility Type</b></label>
    <select id="facility_type">
      <option value="All" selected>All Types</option>
      {facility_type_options}
    </select>

    <button id="applyBtn" style="margin-top:10px;width:100%;padding:8px;font-weight:bold;background:#3f51b5;color:white;border:none;border-radius:5px;cursor:pointer;">Apply</button>
    <hr>
    <p style="font-size:14px;">Move the mouse over any cell to see its stats.<br>Facility locations shown as colored dots.</p>
  </div>
  <div class="MapBody">
    <div id="map"></div>
    <div id="popup" class="ol-popup" style="display:none;"></div>
  </div>
</div>
<script src="https://cdn.jsdelivr.net/npm/ol@v9.2.4/dist/ol.js"></script>
<script>
  var facilities = {json.dumps(facilities, ensure_ascii=False)};
  var popGrid = {json.dumps(pop_grid)};
  var typeColors = {{
    "Hospital": "#D32F2F",
    "PHC": "#388E3C",
    "Pharmacy": "#1976D2",
    "Clinic": "#FFA000",
    "Polyclinic": "#7B1FA2",
    "Lab": "#0288D1",
    "Dental Center": "#C2185B",
    "Imaging Center": "#512DA8"
  }};

  var metric = 'PCNT';
  var minval = 0;
  var selectedFacilityType = 'All';

  var map = new ol.Map({{
    target: 'map',
    layers: [
      new ol.layer.Tile({{ source: new ol.source.OSM() }})
    ],
    view: new ol.View({{
      center: ol.proj.fromLonLat([46.7117, 24.7136]),
      zoom: 11
    }})
  }});

  // --- Population Grid Layer ---
  var popFeatures_all = new ol.format.GeoJSON().readFeatures(popGrid, {{
    featureProjection: 'EPSG:3857',
    dataProjection: 'EPSG:4326'
  }});

  function getColor(value, which) {{
    if (value == null || value === '' || value < minval) return 'rgba(0,0,0,0)';
    var v = Number(value);
    if (which.startsWith("YMED_AGE")) {{
      if (v >= 50) return "#6d0e0e";
      if (v >= 40) return "#b71c1c";
      if (v >= 30) return "#ef5350";
      if (v >= 20) return "#ffb300";
      if (v >= 10) return "#fff176";
      return "#fffde7";
    }} else if (which.includes("DEN")) {{
      if (v > 5000) return "#7f0000";
      if (v > 2000) return "#b71c1c";
      if (v > 1000) return "#ef5350";
      if (v > 500)  return "#ffb300";
      if (v > 200)  return "#fff176";
      return "#fffde7";
    }} else {{
      if (v > 1000) return "#b71c1c";
      if (v > 500)  return "#ef5350";
      if (v > 250)  return "#ffb300";
      if (v > 100)  return "#fff176";
      return "#fffde7";
    }}
  }}

  var popLayer = new ol.layer.Vector({{
    source: new ol.source.Vector(),
    opacity: 0.6
  }});
  map.addLayer(popLayer);

  function styleFeature(f) {{
    var val = f.get(metric);
    return new ol.style.Style({{
      stroke: new ol.style.Stroke({{color: '#666', width: 0.2}}),
      fill: new ol.style.Fill({{color: getColor(val, metric), opacity: 0.5}})
    }});
  }}

  function renderPopLayer() {{
    popLayer.getSource().clear();
    popFeatures_all.forEach(function(f) {{
      var v = f.get(metric);
      if (v != null && v !== '' && Number(v) >= minval)
        f.setStyle(styleFeature(f));
      else
        f.setStyle(new ol.style.Style({{fill: new ol.style.Fill({{color:'rgba(0,0,0,0)'}})}}));
    }});
    popLayer.getSource().addFeatures(popFeatures_all);
  }}

  // Facility Markers (with filtering)
  var facilitySource = new ol.source.Vector();
  var facilityLayer = new ol.layer.Vector({{ source: facilitySource }});
  map.addLayer(facilityLayer);

  function renderFacilities() {{
    facilitySource.clear();
    facilities.forEach(function(fac) {{
      if (selectedFacilityType === "All" || fac.type === selectedFacilityType) {{
        var color = typeColors[fac.type] || "#222";
        var marker = new ol.Feature({{
          geometry: new ol.geom.Point(ol.proj.fromLonLat(fac.coords))
        }});
        marker.setStyle(new ol.style.Style({{
          image: new ol.style.Circle({{
            radius: 7,
            fill: new ol.style.Fill({{color: color}}),
            stroke: new ol.style.Stroke({{color: '#fff', width: 1}})
          }}),
          text: new ol.style.Text({{
            text: fac.name,
            offsetY: -16,
            font: 'bold 10px Arial',
            fill: new ol.style.Fill({{color: "#444"}}),
            stroke: new ol.style.Stroke({{color: "#fff", width: 2}})
          }})
        }}));
        facilitySource.addFeature(marker);
      }}
    }});
  }}

  // Popup for Population Cell Info
  var popup = document.getElementById('popup');
  var overlay = new ol.Overlay({{
    element: popup,
    positioning: 'bottom-center',
    stopEvent: false,
    offset: [0, -8]
  }});
  map.addOverlay(overlay);

  map.on('pointermove', function(evt) {{
    var feature = map.forEachFeatureAtPixel(evt.pixel, function(feature, layer) {{
      if (layer === popLayer) return feature;
    }});
    if (feature) {{
      var p = feature.getProperties();
      var html = "<b>Population:</b> " + (p.PCNT ?? 'N/A') + "<br>" +
                 "<b>Male:</b> " + (p.PM_CNT ?? 'N/A') + "<br>" +
                 "<b>Female:</b> " + (p.PF_CNT ?? 'N/A') + "<br>" +
                 "<b>Density/km²:</b> " + (p.PDEN_KM2 ?? 'N/A') + "<br>" +
                 "<b>Median Age:</b> " + (p.YMED_AGE ?? 'N/A') + "<br>" +
                 "<b>Median Age (M):</b> " + (p.YMED_AGE_M ?? 'N/A') + "<br>" +
                 "<b>Median Age (F):</b> " + (p.YMED_AGE_FM ?? 'N/A');
      popup.innerHTML = html;
      popup.style.display = 'block';
      overlay.setPosition(evt.coordinate);
    }} else {{
      popup.style.display = 'none';
    }}
  }});

  // UI controls
  document.getElementById('applyBtn').onclick = function() {{
    metric = document.getElementById('metric_select').value;
    minval = Number(document.getElementById('minval').value) || 0;
    selectedFacilityType = document.getElementById('facility_type').value;
    renderPopLayer();
    renderFacilities();
  }};
  document.getElementById('facility_type').onchange = function() {{
    selectedFacilityType = this.value;
    renderFacilities();
  }};
  // Initial render
  renderPopLayer();
  renderFacilities();
</script>
</body>
</html>
"""

with open("dashboard_riyadh_population_facilities_filtertype.html", "w", encoding="utf-8") as f:
    f.write(html)

print("dashboard_riyadh_population_facilities_filtertype.html generated! Open it in your browser.")


In [4]:
import pandas as pd
import json

# Load facility data
df = pd.read_excel("random_riyadh_facilities.xlsx")
facilities = []
for _, row in df.iterrows():
    facilities.append({
        "name": row['Name'],
        "coords": [row['Longitude'], row['Latitude']],
        "type": row['Type']
    })

with open("pop_grid_riyadh.geojson", encoding="utf-8") as f:
    pop_grid = json.load(f)

facility_types = sorted(df['Type'].unique())
facility_type_options = "\n".join(
    f'<option value="{ft}">{ft}</option>' for ft in facility_types
)

html = f"""
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <title>Lean Facility Dashboard + Riyadh Population</title>
  <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/ol@v9.2.4/ol.css">
  <style>
    html, body {{
      height: 100%;
      margin: 0;
      padding: 0;
    }}
    body {{
      font-family: sans-serif;
      height: 100%;
      margin: 0;
      padding: 0;
      overflow: hidden;
    }}
    .Container {{
      display: flex;
      height: 100vh;
    }}
    .Sidebar {{
      width: 290px;
      background: #fafaff;
      border-right: 1px solid #ddd;
      padding: 22px 15px 15px 15px;
      box-sizing: border-box;
      font-size: 15px;
    }}
    .Sidebar label, .Sidebar select, .Sidebar input[type=number] {{
      margin-bottom: 12px;
      display: block;
      width: 100%;
    }}
    .MapBody {{
      flex: 1;
      height: 100vh;
      min-width: 300px;
      position: relative;
    }}
    #map {{
      width: 100%;
      height: 100vh;
      min-height: 600px;
      min-width: 400px;
      background: #eee;
    }}
    .ol-popup {{
      position: absolute;
      background-color: white;
      box-shadow: 0 1px 4px rgba(0,0,0,0.2);
      padding: 10px;
      border-radius: 6px;
      border: 1px solid #cccccc;
      min-width: 210px;
      bottom: 12px;
      left: -50px;
      font-size: 14px;
      z-index: 999;
      pointer-events: none;
    }}
    .best-location-star {{
      filter: drop-shadow(0 1px 4px #0008);
    }}
  </style>
</head>
<body>
<div class="Container">
  <div class="Sidebar">
    <h2>Lean</h2>
    <label for="metric_select"><b>Color By Metric</b></label>
    <select id="metric_select">
      <option value="PCNT">Total Population</option>
      <option value="PM_CNT">Male Population</option>
      <option value="PF_CNT">Female Population</option>
      <option value="PDEN_KM2">Density (per km²)</option>
      <option value="YMED_AGE">Median Age</option>
      <option value="YMED_AGE_M">Median Age (M)</option>
      <option value="YMED_AGE_FM">Median Age (F)</option>
    </select>
    <label for="minval"><b>Filter: Show only cells above this value</b></label>
    <input type="number" id="minval" value="0" min="0" step="1">

    <label for="facility_type"><b>Show Facility Type</b></label>
    <select id="facility_type">
      <option value="All" selected>All Types</option>
      {facility_type_options}
    </select>

    <button id="applyBtn" style="margin-top:10px;width:100%;padding:8px;font-weight:bold;background:#3f51b5;color:white;border:none;border-radius:5px;cursor:pointer;">Apply</button>
    <button id="suggestBtn" style="margin-top:7px;width:100%;padding:8px;font-weight:bold;background:#005bbb;color:white;border:none;border-radius:5px;cursor:pointer;">Suggest Best Locations</button>
    <hr>
    <p style="font-size:14px;">Move the mouse over any cell to see its stats.<br>Facility locations shown as colored dots.<br>Best locations will appear as blue stars.</p>
  </div>
  <div class="MapBody">
    <div id="map"></div>
    <div id="popup" class="ol-popup" style="display:none;"></div>
  </div>
</div>
<script src="https://cdn.jsdelivr.net/npm/ol@v9.2.4/dist/ol.js"></script>
<script>
  var facilities = {json.dumps(facilities, ensure_ascii=False)};
  var popGrid = {json.dumps(pop_grid)};
  var typeColors = {{
    "Hospital": "#D32F2F",
    "PHC": "#388E3C",
    "Pharmacy": "#1976D2",
    "Clinic": "#FFA000",
    "Polyclinic": "#7B1FA2",
    "Lab": "#0288D1",
    "Dental Center": "#C2185B",
    "Imaging Center": "#512DA8"
  }};

  var metric = 'PCNT';
  var minval = 0;
  var selectedFacilityType = 'All';
  var bestLocationLayer = null;

  var map = new ol.Map({{
    target: 'map',
    layers: [
      new ol.layer.Tile({{ source: new ol.source.OSM() }})
    ],
    view: new ol.View({{
      center: ol.proj.fromLonLat([46.7117, 24.7136]),
      zoom: 11
    }})
  }});

  // --- Population Grid Layer ---
  var popFeatures_all = new ol.format.GeoJSON().readFeatures(popGrid, {{
    featureProjection: 'EPSG:3857',
    dataProjection: 'EPSG:4326'
  }});

  function getColor(value, which) {{
    if (value == null || value === '' || value < minval) return 'rgba(0,0,0,0)';
    var v = Number(value);
    if (which.startsWith("YMED_AGE")) {{
      if (v >= 50) return "#6d0e0e";
      if (v >= 40) return "#b71c1c";
      if (v >= 30) return "#ef5350";
      if (v >= 20) return "#ffb300";
      if (v >= 10) return "#fff176";
      return "#fffde7";
    }} else if (which.includes("DEN")) {{
      if (v > 5000) return "#7f0000";
      if (v > 2000) return "#b71c1c";
      if (v > 1000) return "#ef5350";
      if (v > 500)  return "#ffb300";
      if (v > 200)  return "#fff176";
      return "#fffde7";
    }} else {{
      if (v > 1000) return "#b71c1c";
      if (v > 500)  return "#ef5350";
      if (v > 250)  return "#ffb300";
      if (v > 100)  return "#fff176";
      return "#fffde7";
    }}
  }}

  var popLayer = new ol.layer.Vector({{
    source: new ol.source.Vector(),
    opacity: 0.6
  }});
  map.addLayer(popLayer);

  function styleFeature(f) {{
    var val = f.get(metric);
    return new ol.style.Style({{
      stroke: new ol.style.Stroke({{color: '#666', width: 0.2}}),
      fill: new ol.style.Fill({{color: getColor(val, metric), opacity: 0.5}})
    }});
  }}

  function renderPopLayer() {{
    popLayer.getSource().clear();
    popFeatures_all.forEach(function(f) {{
      var v = f.get(metric);
      if (v != null && v !== '' && Number(v) >= minval)
        f.setStyle(styleFeature(f));
      else
        f.setStyle(new ol.style.Style({{fill: new ol.style.Fill({{color:'rgba(0,0,0,0)'}})}}));
    }});
    popLayer.getSource().addFeatures(popFeatures_all);
  }}

  // Facility Markers (with filtering)
  var facilitySource = new ol.source.Vector();
  var facilityLayer = new ol.layer.Vector({{ source: facilitySource }});
  map.addLayer(facilityLayer);

  function renderFacilities() {{
    facilitySource.clear();
    facilities.forEach(function(fac) {{
      if (selectedFacilityType === "All" || fac.type === selectedFacilityType) {{
        var color = typeColors[fac.type] || "#222";
        var marker = new ol.Feature({{
          geometry: new ol.geom.Point(ol.proj.fromLonLat(fac.coords))
        }});
        marker.setStyle(new ol.style.Style({{
          image: new ol.style.Circle({{
            radius: 7,
            fill: new ol.style.Fill({{color: color}}),
            stroke: new ol.style.Stroke({{color: '#fff', width: 1}})
          }}),
          text: new ol.style.Text({{
            text: fac.name,
            offsetY: -16,
            font: 'bold 10px Arial',
            fill: new ol.style.Fill({{color: "#444"}}),
            stroke: new ol.style.Stroke({{color: "#fff", width: 2}})
          }})
        }}));
        facilitySource.addFeature(marker);
      }}
    }});
  }}

  // Popup for Population Cell Info
  var popup = document.getElementById('popup');
  var overlay = new ol.Overlay({{
    element: popup,
    positioning: 'bottom-center',
    stopEvent: false,
    offset: [0, -8]
  }});
  map.addOverlay(overlay);

  map.on('pointermove', function(evt) {{
    var feature = map.forEachFeatureAtPixel(evt.pixel, function(feature, layer) {{
      if (layer === popLayer) return feature;
    }});
    if (feature) {{
      var p = feature.getProperties();
      var html = "<b>Population:</b> " + (p.PCNT ?? 'N/A') + "<br>" +
                 "<b>Male:</b> " + (p.PM_CNT ?? 'N/A') + "<br>" +
                 "<b>Female:</b> " + (p.PF_CNT ?? 'N/A') + "<br>" +
                 "<b>Density/km²:</b> " + (p.PDEN_KM2 ?? 'N/A') + "<br>" +
                 "<b>Median Age:</b> " + (p.YMED_AGE ?? 'N/A') + "<br>" +
                 "<b>Median Age (M):</b> " + (p.YMED_AGE_M ?? 'N/A') + "<br>" +
                 "<b>Median Age (F):</b> " + (p.YMED_AGE_FM ?? 'N/A');
      popup.innerHTML = html;
      popup.style.display = 'block';
      overlay.setPosition(evt.coordinate);
    }} else {{
      popup.style.display = 'none';
    }}
  }});

  // UI controls
  document.getElementById('applyBtn').onclick = function() {{
    metric = document.getElementById('metric_select').value;
    minval = Number(document.getElementById('minval').value) || 0;
    selectedFacilityType = document.getElementById('facility_type').value;
    renderPopLayer();
    renderFacilities();
    if (bestLocationLayer) {{
      bestLocationLayer.getSource().clear();
    }}
  }};
  document.getElementById('facility_type').onchange = function() {{
    selectedFacilityType = this.value;
    renderFacilities();
    if (bestLocationLayer) {{
      bestLocationLayer.getSource().clear();
    }}
  }};
  // Initial render
  renderPopLayer();
  renderFacilities();

  // -- Suggest Best Locations --
  function haversineDistance(lon1, lat1, lon2, lat2) {{
    function toRad(x) {{ return x * Math.PI / 180; }}
    var R = 6371000;
    var dLat = toRad(lat2-lat1);
    var dLon = toRad(lon2-lon1);
    var a = Math.sin(dLat/2)*Math.sin(dLat/2) +
      Math.cos(toRad(lat1))*Math.cos(toRad(lat2)) *
      Math.sin(dLon/2)*Math.sin(dLon/2);
    var c = 2*Math.atan2(Math.sqrt(a),Math.sqrt(1-a));
    return R*c;
  }}

  document.getElementById('suggestBtn').onclick = function() {{
    if (bestLocationLayer) {{
      bestLocationLayer.getSource().clear();
    }} else {{
      bestLocationLayer = new ol.layer.Vector({{
        source: new ol.source.Vector()
      }});
      map.addLayer(bestLocationLayer);
    }}

    var selectedFacilities = facilities.filter(function(fac) {{
      return selectedFacilityType === "All" || fac.type === selectedFacilityType;
    }});

    var candidates = [];
    popFeatures_all.forEach(function(f) {{
      var v = f.get(metric);
      if (v != null && v !== '' && Number(v) >= minval) {{
        var geom = f.getGeometry();
        var coord = ol.proj.toLonLat(geom.getClosestPoint(geom.getExtent()));
        var minDist = Infinity;
        selectedFacilities.forEach(function(fac) {{
          var d = haversineDistance(coord[0], coord[1], fac.coords[0], fac.coords[1]);
          if (d < minDist) minDist = d;
        }});
        candidates.push({{ coord: coord, value: Number(v), minDist: minDist }});
      }}
    }});

    // Improved scoring: weighted combination
    var maxValue = Math.max(...candidates.map(c => c.value));
    var maxDist = Math.max(...candidates.map(c => c.minDist));
    if (maxValue == 0) maxValue = 1;
    if (maxDist == 0) maxDist = 1;
    candidates.forEach(function(c) {{
      c.score = 0.7 * (c.value / maxValue) + 0.3 * (c.minDist / maxDist);
    }});
    candidates.sort(function(a, b) {{
      return b.score - a.score;
    }});

    var top3 = [];
    var minDistanceMeters = 1000;
    for (var i = 0; i < candidates.length && top3.length < 3; i++) {{
      var c = candidates[i];
      if (top3.every(function(other) {{
        return haversineDistance(c.coord[0], c.coord[1], other.coord[0], other.coord[1]) > minDistanceMeters;
      }})) {{
        top3.push(c);
      }}
    }}

    var starSVG = `
      <svg xmlns='http://www.w3.org/2000/svg' width='44' height='44'>
        <polygon points='22,2 27,16 42,16 29,25 34,39 22,30 10,39 15,25 2,16 17,16'
          style='fill:#005bbb;stroke:#fff;stroke-width:2;'/>
      </svg>`;
    var starStyle = new ol.style.Style({{
      image: new ol.style.Icon({{
        src: "data:image/svg+xml;utf8," + encodeURIComponent(starSVG),
        scale: 1,
        className: "best-location-star"
      }})
    }});
    var src = bestLocationLayer.getSource();
    top3.forEach(function(c, idx) {{
      var f = new ol.Feature({{
        geometry: new ol.geom.Point(ol.proj.fromLonLat(c.coord))
      }});
      f.setStyle(starStyle);
      src.addFeature(f);
    }});
    if (top3.length === 0) {{
      alert("No suitable locations found!");
    }}
  }};
</script>
</body>
</html>
"""

with open("dashboard_riyadh_population_bestlocations.html", "w", encoding="utf-8") as f:
    f.write(html)

print("dashboard_riyadh_population_bestlocations.html generated! Open it in your browser.")


dashboard_riyadh_population_bestlocations.html generated! Open it in your browser.


In [39]:
import pandas as pd
import json

# Load facility data
df = pd.read_excel("random_riyadh_facilities.xlsx")
facilities = []
for _, row in df.iterrows():
    facilities.append({
        "name": row['Name'],
        "coords": [row['Longitude'], row['Latitude']],
        "type": row['Type']
    })

with open("pop_grid_riyadh.geojson", encoding="utf-8") as f:
    pop_grid = json.load(f)

facility_types = sorted(df['Type'].unique())
facility_type_options = "\n".join(
    f'<option value="{ft}">{ft}</option>' for ft in facility_types
)

html = f"""
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <title>Lean Facility Dashboard + Riyadh Population</title>
  <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/ol@v9.2.4/ol.css">
  <style>
    html, body {{
      height: 100%;
      margin: 0;
      padding: 0;
    }}
    body {{
      font-family: sans-serif;
      height: 100%;
      margin: 0;
      padding: 0;
      overflow: hidden;
    }}
    .Container {{
      display: flex;
      height: 100vh;
    }}
    .Sidebar {{
      width: 290px;
      background: #fafaff;
      border-right: 1px solid #ddd;
      padding: 22px 15px 15px 15px;
      box-sizing: border-box;
      font-size: 15px;
    }}
    .Sidebar label, .Sidebar select, .Sidebar input[type=number] {{
      margin-bottom: 12px;
      display: block;
      width: 100%;
    }}
    .MapBody {{
      flex: 1;
      height: 100vh;
      min-width: 300px;
      position: relative;
    }}
    #map {{
      width: 100%;
      height: 100vh;
      min-height: 600px;
      min-width: 400px;
      background: #eee;
    }}
    .ol-popup {{
      position: absolute;
      background-color: white;
      box-shadow: 0 1px 4px rgba(0,0,0,0.2);
      padding: 10px;
      border-radius: 6px;
      border: 1px solid #cccccc;
      min-width: 210px;
      bottom: 12px;
      left: -50px;
      font-size: 14px;
      z-index: 999;
      pointer-events: none;
    }}
    .best-location-star {{
      filter: drop-shadow(0 1px 4px #0008);
    }}
  </style>
</head>
<body>
<div class="Container">
  <div class="Sidebar">
    <h2>Lean</h2>
    <label for="metric_select"><b>Color By Metric</b></label>
    <select id="metric_select">
      <option value="PCNT">Total Population</option>
      <option value="PM_CNT">Male Population</option>
      <option value="PF_CNT">Female Population</option>
      <option value="PDEN_KM2">Density (per km²)</option>
      <option value="YMED_AGE">Median Age</option>
      <option value="YMED_AGE_M">Median Age (M)</option>
      <option value="YMED_AGE_FM">Median Age (F)</option>
    </select>
    <label for="minval"><b>Filter: Show only cells above this value</b></label>
    <input type="number" id="minval" value="0" min="0" step="1">

    <label for="facility_type"><b>Show Facility Type</b></label>
    <select id="facility_type">
      <option value="All" selected>All Types</option>
      {facility_type_options}
    </select>

    <button id="applyBtn" style="margin-top:10px;width:100%;padding:8px;font-weight:bold;background:#3f51b5;color:white;border:none;border-radius:5px;cursor:pointer;">Apply</button>
    <button id="suggestBtn" style="margin-top:7px;width:100%;padding:8px;font-weight:bold;background:#005bbb;color:white;border:none;border-radius:5px;cursor:pointer;">Suggest Best Locations</button>
    <hr>
    <p style="font-size:14px;">Move the mouse over any cell to see its stats.<br>Facility locations shown as colored dots.<br>Best locations will appear as blue stars.<br>Hover over a blue star to see details!</p>
  </div>
  <div class="MapBody">
    <div id="map"></div>
    <div id="popup" class="ol-popup" style="display:none;"></div>
  </div>
</div>
<script src="https://cdn.jsdelivr.net/npm/ol@v9.2.4/dist/ol.js"></script>
<script>
  var facilities = {json.dumps(facilities, ensure_ascii=False)};
  var popGrid = {json.dumps(pop_grid)};
  var typeColors = {{
    "Hospital": "#D32F2F",
    "PHC": "#388E3C",
    "Pharmacy": "#1976D2",
    "Clinic": "#FFA000",
    "Polyclinic": "#7B1FA2",
    "Lab": "#0288D1",
    "Dental Center": "#C2185B",
    "Imaging Center": "#512DA8"
  }};

  var metric = 'PCNT';
  var minval = 0;
  var selectedFacilityType = 'All';
  var bestLocationLayer = null;

  var map = new ol.Map({{
    target: 'map',
    layers: [
      new ol.layer.Tile({{ source: new ol.source.OSM() }})
    ],
    view: new ol.View({{
      center: ol.proj.fromLonLat([46.7117, 24.7136]),
      zoom: 11
    }})
  }});

  // --- Population Grid Layer ---
  var popFeatures_all = new ol.format.GeoJSON().readFeatures(popGrid, {{
    featureProjection: 'EPSG:3857',
    dataProjection: 'EPSG:4326'
  }});

  function getColor(value, which) {{
    if (value == null || value === '' || value < minval) return 'rgba(0,0,0,0)';
    var v = Number(value);
    if (which.startsWith("YMED_AGE")) {{
      if (v >= 50) return "#6d0e0e";
      if (v >= 40) return "#b71c1c";
      if (v >= 30) return "#ef5350";
      if (v >= 20) return "#ffb300";
      if (v >= 10) return "#fff176";
      return "#fffde7";
    }} else if (which.includes("DEN")) {{
      if (v > 5000) return "#7f0000";
      if (v > 2000) return "#b71c1c";
      if (v > 1000) return "#ef5350";
      if (v > 500)  return "#ffb300";
      if (v > 200)  return "#fff176";
      return "#fffde7";
    }} else {{
      if (v > 1000) return "#b71c1c";
      if (v > 500)  return "#ef5350";
      if (v > 250)  return "#ffb300";
      if (v > 100)  return "#fff176";
      return "#fffde7";
    }}
  }}

  var popLayer = new ol.layer.Vector({{
    source: new ol.source.Vector(),
    opacity: 0.6
  }});
  map.addLayer(popLayer);

  function styleFeature(f) {{
    var val = f.get(metric);
    return new ol.style.Style({{
      stroke: new ol.style.Stroke({{color: '#666', width: 0.2}}),
      fill: new ol.style.Fill({{color: getColor(val, metric), opacity: 0.5}})
    }});
  }}

  function renderPopLayer() {{
    popLayer.getSource().clear();
    popFeatures_all.forEach(function(f) {{
      var v = f.get(metric);
      if (v != null && v !== '' && Number(v) >= minval)
        f.setStyle(styleFeature(f));
      else
        f.setStyle(new ol.style.Style({{fill: new ol.style.Fill({{color:'rgba(0,0,0,0)'}})}}));
    }});
    popLayer.getSource().addFeatures(popFeatures_all);
  }}

  // Facility Markers (with filtering)
  var facilitySource = new ol.source.Vector();
  var facilityLayer = new ol.layer.Vector({{ source: facilitySource }});
  map.addLayer(facilityLayer);

  function renderFacilities() {{
    facilitySource.clear();
    facilities.forEach(function(fac) {{
      if (selectedFacilityType === "All" || fac.type === selectedFacilityType) {{
        var color = typeColors[fac.type] || "#222";
        var marker = new ol.Feature({{
          geometry: new ol.geom.Point(ol.proj.fromLonLat(fac.coords))
        }});
        marker.setStyle(new ol.style.Style({{
          image: new ol.style.Circle({{
            radius: 7,
            fill: new ol.style.Fill({{color: color}}),
            stroke: new ol.style.Stroke({{color: '#fff', width: 1}})
          }}),
          text: new ol.style.Text({{
            text: fac.name,
            offsetY: -16,
            font: 'bold 10px Arial',
            fill: new ol.style.Fill({{color: "#444"}}),
            stroke: new ol.style.Stroke({{color: "#fff", width: 2}})
          }})
        }}));
        facilitySource.addFeature(marker);
      }}
    }});
  }}

  // Popup for Population Cell Info
  var popup = document.getElementById('popup');
  var overlay = new ol.Overlay({{
    element: popup,
    positioning: 'bottom-center',
    stopEvent: false,
    offset: [0, -8]
  }});
  map.addOverlay(overlay);

  map.on('pointermove', function(evt) {{
    var found = false;
    // First, check if hovering over a blue star (best location)
    map.forEachFeatureAtPixel(evt.pixel, function(feature, layer) {{
      if (bestLocationLayer && layer === bestLocationLayer && feature.get("best_info")) {{
        var info = feature.get("best_info");
        popup.innerHTML =
          `<b>Suggested New Location</b><br>
          <b>` + info.metric + `</b>: ` + info.value.toLocaleString() + `<br>
          <b>Distance to Nearest Facility:</b> ` + Math.round(info.minDist) + ` m<br>
          <b>Composite Score:</b> ` + info.score.toFixed(3) + `<br>
          <span style="font-size:11px;color:#888;">[` + info.coord.map(x=>x.toFixed(5)).join(", ") + `]</span>`;
        popup.style.display = 'block';
        overlay.setPosition(evt.coordinate);
        found = true;
      }}
    }});
    if (!found) {{
      // Otherwise, show population cell info
      var feature = map.forEachFeatureAtPixel(evt.pixel, function(feature, layer) {{
        if (layer === popLayer) return feature;
      }});
      if (feature) {{
        var p = feature.getProperties();
        var html = "<b>Population:</b> " + (p.PCNT ?? 'N/A') + "<br>" +
                  "<b>Male:</b> " + (p.PM_CNT ?? 'N/A') + "<br>" +
                  "<b>Female:</b> " + (p.PF_CNT ?? 'N/A') + "<br>" +
                  "<b>Density/km²:</b> " + (p.PDEN_KM2 ?? 'N/A') + "<br>" +
                  "<b>Median Age:</b> " + (p.YMED_AGE ?? 'N/A') + "<br>" +
                  "<b>Median Age (M):</b> " + (p.YMED_AGE_M ?? 'N/A') + "<br>" +
                  "<b>Median Age (F):</b> " + (p.YMED_AGE_FM ?? 'N/A');
        popup.innerHTML = html;
        popup.style.display = 'block';
        overlay.setPosition(evt.coordinate);
      }} else {{
        popup.style.display = 'none';
      }}
    }}
  }});

  // UI controls
  document.getElementById('applyBtn').onclick = function() {{
    metric = document.getElementById('metric_select').value;
    minval = Number(document.getElementById('minval').value) || 0;
    selectedFacilityType = document.getElementById('facility_type').value;
    renderPopLayer();
    renderFacilities();
    if (bestLocationLayer) {{
      bestLocationLayer.getSource().clear();
    }}
  }};
  document.getElementById('facility_type').onchange = function() {{
    selectedFacilityType = this.value;
    renderFacilities();
    if (bestLocationLayer) {{
      bestLocationLayer.getSource().clear();
    }}
  }};
  // Initial render
  renderPopLayer();
  renderFacilities();

  // -- Suggest Best Locations with hover info --
  function haversineDistance(lon1, lat1, lon2, lat2) {{
    function toRad(x) {{ return x * Math.PI / 180; }}
    var R = 6371000;
    var dLat = toRad(lat2-lat1);
    var dLon = toRad(lon2-lon1);
    var a = Math.sin(dLat/2)*Math.sin(dLat/2) +
      Math.cos(toRad(lat1))*Math.cos(toRad(lat2)) *
      Math.sin(dLon/2)*Math.sin(dLon/2);
    var c = 2*Math.atan2(Math.sqrt(a),Math.sqrt(1-a));
    return R*c;
  }}

  document.getElementById('suggestBtn').onclick = function() {{
    if (bestLocationLayer) {{
      bestLocationLayer.getSource().clear();
    }} else {{
      bestLocationLayer = new ol.layer.Vector({{
        source: new ol.source.Vector()
      }});
      map.addLayer(bestLocationLayer);
    }}

    var selectedFacilities = facilities.filter(function(fac) {{
      return selectedFacilityType === "All" || fac.type === selectedFacilityType;
    }});

    var candidates = [];
    popFeatures_all.forEach(function(f) {{
      var v = f.get(metric);
      if (v != null && v !== '' && Number(v) >= minval) {{
        var geom = f.getGeometry();
        var coord = ol.proj.toLonLat(geom.getClosestPoint(geom.getExtent()));
        var minDist = Infinity;
        selectedFacilities.forEach(function(fac) {{
          var d = haversineDistance(coord[0], coord[1], fac.coords[0], fac.coords[1]);
          if (d < minDist) minDist = d;
        }});
        candidates.push({{ coord: coord, value: Number(v), minDist: minDist }});
      }}
    }});

    var maxValue = Math.max(...candidates.map(c => c.value));
    var maxDist = Math.max(...candidates.map(c => c.minDist));
    if (maxValue == 0) maxValue = 1;
    if (maxDist == 0) maxDist = 1;
    candidates.forEach(function(c) {{
      c.score = 0.7 * (c.value / maxValue) + 0.3 * (c.minDist / maxDist);
    }});
    candidates.sort(function(a, b) {{
      return b.score - a.score;
    }});

    var top3 = [];
    var minDistanceMeters = 1000;
    for (var i = 0; i < candidates.length && top3.length < 3; i++) {{
      var c = candidates[i];
      if (top3.every(function(other) {{
        return haversineDistance(c.coord[0], c.coord[1], other.coord[0], other.coord[1]) > minDistanceMeters;
      }})) {{
        top3.push(c);
      }}
    }}

    var starSVG = `
      <svg xmlns='http://www.w3.org/2000/svg' width='44' height='44'>
        <polygon points='22,2 27,16 42,16 29,25 34,39 22,30 10,39 15,25 2,16 17,16'
          style='fill:#005bbb;stroke:#fff;stroke-width:2;'/>
      </svg>`;
    var starStyle = new ol.style.Style({{
      image: new ol.style.Icon({{
        src: "data:image/svg+xml;utf8," + encodeURIComponent(starSVG),
        scale: 1,
        className: "best-location-star"
      }})
    }});
    var src = bestLocationLayer.getSource();
    top3.forEach(function(c, idx) {{
      var f = new ol.Feature({{
        geometry: new ol.geom.Point(ol.proj.fromLonLat(c.coord))
      }});
      f.set("best_info", {{
        value: c.value,
        minDist: c.minDist,
        score: c.score,
        metric: metric,
        coord: c.coord
      }});
      f.setStyle(starStyle);
      src.addFeature(f);
    }});
    if (top3.length === 0) {{
      alert("No suitable locations found!");
    }}
  }};
</script>
</body>
</html>
"""

with open("dashboard_riyadh_population_bestlocations_tooltip.html", "w", encoding="utf-8") as f:
    f.write(html)

print("dashboard_riyadh_population_bestlocations_tooltip.html generated! Open it in your browser.")


dashboard_riyadh_population_bestlocations_tooltip.html generated! Open it in your browser.


In [1]:
!streamlit run app.py


^C


In [ ]:
import folium
import json
import pandas as pd

# Load population GeoJSON
with open("/mnt/data/pop_grid_riyadh.geojson", encoding="utf-8") as f:
    pop_data = json.load(f)

# Create a folium map centered on Riyadh
m = folium.Map(location=[24.7136, 46.6753], zoom_start=10, tiles="cartodbpositron")

# Add a choropleth layer for population
choropleth = folium.Choropleth(
    geo_data=pop_data,
    name="Population Density",
    data=pd.DataFrame([
        {
            "MAIN_ID": feature["properties"]["MAIN_ID"],
            "PDEN_KM2": feature["properties"]["PDEN_KM2"]
        }
        for feature in pop_data["features"]
    ]),
    columns=["MAIN_ID", "PDEN_KM2"],
    key_on="feature.properties.MAIN_ID",
    fill_color="YlOrRd",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Population Density (per km²)"
).add_to(m)

# Add tooltips
folium.GeoJson(
    pop_data,
    name="Population Info",
    style_function=lambda x: {"fillOpacity": 0},
    tooltip=folium.features.GeoJsonTooltip(fields=["MAIN_ID", "PCNT", "PM_CNT", "PF_CNT", "PDEN_KM2", "YMED_AGE"])
).add_to(m)

# Save and show map
map_path = "/mnt/data/choropleth_population_map.html"
m.save(map_path)
map_path


FileNotFoundError: [Errno 2] No such file or directory: '/mnt/data/pop_grid_riyadh.geojson'

: 

In [ ]:
pip install streamlit



SyntaxError: invalid syntax (2334867191.py, line 1)

: 